# calculate ACC and NMAE within each LME along the vertical direction

In [1]:
import matplotlib.pyplot as plt
import xarray as xr
import netCDF4
import numpy as np
from scipy.stats import pearsonr
from matplotlib import cm,colors,gridspec
import scipy.signal as sgl
import os

In [2]:
USER = os.environ['USER']

In [3]:
#varr = 'O2'
varr = 'AOU'
#varr = 'TEMP'
#varr = 'SALT'

## FOSI hindcast 

In [4]:
datadir = f'/glade/scratch/{USER}/DPLE-results'
filename = f'{datadir}/DPLE-FOSI_hindcast_Phi_Eo_space_horizontal_mean_LME_1948-2017_yearly.nc'
ds_hind = xr.open_dataset(filename)
ds_hind

<xarray.Dataset>
Dimensions:  (lme: 11, z_t: 60, year: 70)
Coordinates:
  * lme      (lme) int64 1 2 3 4 5 6 7 8 9 10 65
  * z_t      (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.125e+05 5.375e+05
  * year     (year) int64 0 1 2 3 4 5 6 7 8 9 ... 60 61 62 63 64 65 66 67 68 69
Data variables: (12/13)
    Phi_0    (lme, year, z_t) float64 ...
    Phi_1    (lme, year, z_t) float64 ...
    Phi_2    (lme, year, z_t) float64 ...
    Phi_3    (lme, year, z_t) float64 ...
    Phi_4    (lme, year, z_t) float64 ...
    Phi_5    (lme, year, z_t) float64 ...
    ...       ...
    Phi_7    (lme, year, z_t) float64 ...
    Phi_8    (lme, year, z_t) float64 ...
    Phi_9    (lme, year, z_t) float64 ...
    Phi_10   (lme, year, z_t) float64 ...
    Phi_11   (lme, year, z_t) float64 ...
    Phi_12   (lme, year, z_t) float64 ...

## DPLE forecast

In [5]:
filename = f'{datadir}/DPLE_driftcorrected_Phi_components_{varr}_Eo_space_horizontal_mean_LME11_ens_mean_yearly.nc'
ds_fore = xr.open_dataset(filename)
ds_fore

<xarray.Dataset>
Dimensions:    (lme: 11, Y: 64, z_t: 35, year: 10)
Coordinates:
  * lme        (lme) int64 1 2 3 4 5 6 7 8 9 10 65
  * Y          (Y) int64 1954 1955 1956 1957 1958 ... 2013 2014 2015 2016 2017
  * z_t        (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.277e+04 5.794e+04
  * year       (year) int64 0 1 2 3 4 5 6 7 8 9
Data variables:
    Phi_AOU_0  (lme, Y, year, z_t) float64 ...
    Phi_AOU_1  (lme, Y, year, z_t) float64 ...
    Phi_AOU_2  (lme, Y, year, z_t) float64 ...

## Calculate Corr/Rcri Matrix for Lead Year v. LME box#

In [6]:
var = 'Phi'

In [7]:
import tools
alpha = 0.05
# time period for evaluation 1954-2007
yr1 = 1954
yr2 = 2007
# lead year range: 1,2,...,9,10
ldyrs = range(1, 11)
#
nver = range(35)
# persistence ACC: var1; var2
# DPLE ACC:        var2; var3
lme = [1,2,3,4,5,6,7,8,9,10,65]

lmen = ['EBS','GoA','CC','GoC','GoM','SEUS','NEUS','SS','LN','IPH','AI']
# 9/13 phis
#nphi = 9
#nphi = 13
nphi = 3
#
corr_p = np.ma.zeros([nphi, len(lme), len(ldyrs), len(nver)]); corr_p.mask=True
rcri_p = np.ma.zeros([nphi, len(lme), len(ldyrs), len(nver)]); rcri_p.mask=True
corr_d = corr_p.copy()
rcri_d = rcri_p.copy()
corr13 = corr_p.copy()
rcri13 = rcri_p.copy()
nmae_p = corr_p.copy()
nmae_d = corr_p.copy()
#
for cc,vv in [[0,0],[1,6],[2,12]]:
    for nn in range(len(lme)):
        box = lme[nn]
        var1 = ds_hind[f'{var}_{vv}'].sel(lme=box).isel(z_t=range(35))[yr1-1948:yr2-1948+1].values
        var1c = np.tile(np.nanmean(var1,axis=0)[np.newaxis,:],[yr2-yr1+1,1])
        var1 = var1 - var1c
        #
        for lead in range(1, 11):
            var2 = ds_hind[f'{var}_{vv}'].sel(lme=box).isel(z_t=range(35))[yr1-1948+lead:yr2-1948+1+lead].values
            var2c = np.tile(np.nanmean(var2,axis=0)[np.newaxis,:],[yr2-yr1+1,1])
            var2 = var2 - var2c
            var3 = ds_fore[f'{var}_{varr}_{vv}'].sel(lme=box)[yr1-1954:yr2-1954+1, lead-1].values
            var3c = np.tile(np.nanmean(var3,axis=0)[np.newaxis,:],[yr2-yr1+1,1])
            var3 = var3 - var3c
            #
            for zz in range(35):
                # mask nan values
                mask1 = np.isnan(var1[:,zz])
                mask2 = np.isnan(var2[:,zz])
                mask3 = np.isnan(var3[:,zz])
                mem1 = np.ma.array(var1[:,zz], mask=mask1)
                mem2 = np.ma.array(var2[:,zz], mask=mask2)
                mem3 = np.ma.array(var3[:,zz], mask=mask3)
                # Persistence ACC & NMAE
                fmask = np.logical_and(mem1.mask, mem2.mask)
                if np.sum(fmask) != len(fmask):
                    nen1 = mem1[~fmask]
                    nen2 = mem2[~fmask]
                    corr_p[cc, nn, lead-1, zz] = pearsonr(nen1, nen2)[0]
                    neff = tools.neff3(nen1, nen2, len(nen1))
                    rcri_p[cc, nn, lead-1, zz] = tools.calculate_parson_corr_critical_value(neff, alpha)
                    #
                    nmae_p[cc, nn, lead-1, zz] = np.nanmean(np.abs(nen1-nen2)/np.nanstd(nen2))
                # DPLE ACC & NMAE
                fmask = np.logical_and(mem2.mask, mem3.mask)
                if np.sum(fmask) != len(fmask):
                    nen2 = mem2[~fmask]
                    nen3 = mem3[~fmask]
                    corr_d[cc, nn, lead-1, zz] = pearsonr(nen2, nen3)[0]
                    neff = tools.neff3(nen2, nen3, len(nen2))
                    rcri_d[cc, nn, lead-1, zz] = tools.calculate_parson_corr_critical_value(neff, alpha)
                    #
                    nmae_d[cc, nn, lead-1, zz] = np.nanmean(np.abs(nen3-nen2)/np.nanstd(nen2))
                # Correlation for ACC Difference Significance Test
                fmask = np.logical_and(mem1.mask, mem3.mask)
                if np.sum(fmask) != len(fmask):
                    nen1 = mem1[~fmask]
                    nen3 = mem3[~fmask]
                    corr13[cc, nn, lead-1, zz] = pearsonr(nen1, nen3)[0]
                    neff = tools.neff3(nen1, nen3, len(nen1))
                    rcri13[cc, nn, lead-1, zz] = tools.calculate_parson_corr_critical_value(neff, alpha)
        #

In [8]:
data = xr.Dataset({"corr_p": (("nphi","nbox","nlead","nver"),corr_p),\
                   "rcri_p": (("nphi","nbox","nlead","nver"),rcri_p),\
                   "corr_d": (("nphi","nbox","nlead","nver"),corr_d),\
                   "rcri_d": (("nphi","nbox","nlead","nver"),rcri_d),\
                   "corr13": (("nphi","nbox","nlead","nver"),corr13),\
                   "rcri13": (("nphi","nbox","nlead","nver"),rcri13),\
                   "nmae_p": (("nphi","nbox","nlead","nver"),nmae_p),\
                   "nmae_d": (("nphi","nbox","nlead","nver"),nmae_d)})

In [9]:
dout = f'/glade/scratch/{USER}/DPLE-results'
data.load()

<xarray.Dataset>
Dimensions:  (nphi: 3, nbox: 11, nlead: 10, nver: 35)
Dimensions without coordinates: nphi, nbox, nlead, nver
Data variables:
    corr_p   (nphi, nbox, nlead, nver) float64 0.4832 0.4936 ... -0.1035 -0.12
    rcri_p   (nphi, nbox, nlead, nver) float64 0.344 0.344 0.3494 ... 0.95 0.95
    corr_d   (nphi, nbox, nlead, nver) float64 -0.1285 -0.1572 ... 0.3285 0.3237
    rcri_d   (nphi, nbox, nlead, nver) float64 0.2845 0.2787 ... 0.6664 0.7067
    corr13   (nphi, nbox, nlead, nver) float64 -0.3392 -0.3553 ... -0.005745
    rcri13   (nphi, nbox, nlead, nver) float64 0.2845 0.2787 ... 0.7067 0.7067
    nmae_p   (nphi, nbox, nlead, nver) float64 0.8363 0.8239 ... 1.274 1.287
    nmae_d   (nphi, nbox, nlead, nver) float64 0.8051 0.8068 ... 0.7259 0.7497

In [10]:
data.to_netcdf(f'{dout}/DPLE_LME_box01-11_ACC_NMAE_{var}_components_{varr}_Eo_space_horizontal_mean_leadyear_1-10.nc', mode='w')